In [51]:
import numpy as np

# time the while loop;
from datetime import timedelta
import time

In [52]:
# proposition langage des mathématiques

# aventurier.position(X, Y)
# aventurier.etat(E)
# joyau.position(X, Y)

# map = [
    # [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    # [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    # [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    # [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    # [1, 1, 0, 0, 0, 0, 0, 0, 1, 1],
    # [1, 1, 0, 1, 1, 1, 1, 1, 1, 1],
    # [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    # [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    # [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
# ]

moves = [(0, +1), (0, -1),
     (+1, 0), (-1, 0)]

adventurer = dict()
jewel = dict()

pits = dict()
pits["representation"] = "P"
pits["inst"] = list()
pits["inst"].append([5, 7])

wumpus = "W"

adventurer["representation"] = "A"
jewel["representation"] = "*"

In [86]:
# initial state => 0 till the jewel is found (not used?)
adventurer["state"] = 0

adventurer["position"] = [6, 0]

# map 1 and 2
# jewel["position"] = [4, 7]

# map 3
# jewel["position"] = [1, 8]

# map -- td5
jewel["position"] = [0, 5]

# the content of my file
with open("labyrinth4.txt") as f:
    # .strip() to remove the '\n' character
    my_file = [line.strip() for line in f.readlines()]

# raw, col => dimensions of the labyrinth
def print_map(raw, col):
    # filled the matrix with ""
    field = np.full((10, 10), "")
    for i in range(raw):
        for j in range(col):
            if my_file[i][j] == '1':
                field[i][j] = 'X'
            else:
                field[i][j] = '.'
            if i == jewel["position"][0] and j == jewel["position"][1]:
                field[i][j] = jewel["representation"]
            if i == adventurer["position"][0] and j == adventurer["position"][1]:
                field[i][j] = adventurer["representation"]
            # adventurer is overwritten by a pit or wumpus symbol
            if [i, j] in pits["inst"]:
                field[i][j] = pits["representation"]
            if i == 8 and j == 4:
                field[i][j] = wumpus
    return field

matrix = print_map(10, 10)
for l in matrix:
    print(l)

['X' 'X' 'X' 'X' 'X' '*' 'X' 'X' 'X' 'X']
['X' 'X' 'X' '.' 'X' '.' '.' '.' '.' 'X']
['X' 'X' '.' '.' '.' '.' 'X' 'X' '.' 'X']
['X' 'X' '.' 'X' 'X' 'X' 'X' '.' '.' 'X']
['X' 'X' '.' '.' '.' '.' '.' '.' 'X' 'X']
['X' 'X' '.' 'X' 'X' '.' 'X' 'P' 'X' 'X']
['A' '.' '.' '.' 'X' '.' 'X' '.' 'X' 'X']
['X' 'X' 'X' '.' 'X' 'X' 'X' '.' '.' 'X']
['X' 'X' 'X' '.' 'W' '.' 'X' 'X' '.' 'X']
['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']


In [91]:
# breeze and smell cells;
danger_cells = list()

def hints(raw, col):
    for i in range(raw):
        for j in range(col):
            if [i, j] in pits["inst"] or [i, j] == [8, 4]:
                [danger_cells.append(x) for x in get_positions_possibles([i, j], moves)]

# the rule 'cannot go in the wall' is not applied
def get_positions_possibles(ad_pos, moves_array):
    new_pos = list()
    for x,y in moves_array:
        new_pos.append([x+ad_pos[0], y+ad_pos[1]])
    return new_pos

# PITS AND WUMPUS ARE NOT BLOCKED HERE
def is_allowed(pos, field):
    # if the index is negatif or outside of the field;
    if pos[0] < 0 or pos[0] >= len(field) or pos[1] >= len(field[0]) or pos[1] < 0:
        return False
    elif field[pos[0]][pos[1]] == 'X':
        return False
    else:
        return True

hints(10, 10)
print(danger_cells)

[[5, 8], [5, 6], [6, 7], [4, 7], [8, 5], [8, 3], [9, 4], [7, 4]]


In [92]:
to_explore = list()
already_explored = list()

def reorganise():
    counter = 0
    print(danger_cells)
    for i in range(len(to_explore)):
        if to_explore[i] not in danger_cells:
            counter += 1
        else:
            pos = to_explore.pop(i)
            to_explore.append(pos)
    return counter

def prochaine_pos(posArray):
    for pos in posArray:
        # positions are explored ONCE
        if pos not in already_explored and pos not in to_explore:
            if is_allowed(pos, matrix):
                # .append() -> choice of LIFO
                to_explore.append(pos)
                
    sure_cells_num = reorganise()
    print(sure_cells_num)
    print(to_explore)
    if len(to_explore) > 1:
        if sure_cells_num > 0:
            # no possibility to explore more than 1 safe way (u cannot go back)
            to_explore.pop(np.random.randint(0, sure_cells_num))
        # no safe ways are available
        # trap scenario
        else:
            to_explore.pop(np.random.randint(0, len(to_explore)))
    return 0

def main():
    # initial state of the adventurer
    to_explore.append([6, 0])
    user_input = input("Type in enter to continue or Q to quit:\n")
    starttime = time.perf_counter()
    while user_input.upper() != "Q" and len(to_explore) != 0:
        print("position currently being explored -> ", to_explore[0])
        current = to_explore.pop(0)
        adventurer["position"] = current
        
        prochaine_pos(get_positions_possibles(current, moves))
        already_explored.append(current)
        
        matrix = print_map(10, 10)
        [print(l) for l in matrix]
        if adventurer["position"] == jewel["position"]:
            print("You found the jewel. Congrats!")
            break
        if adventurer["position"] in pits["inst"] or adventurer["position"] == [8, 4]:
            print("You got in a pit or got eaten by a wumpus.")
            break
        user_input = input("Keep going!\n")
    if user_input == "q":
        print("You ended the game.")
    elif adventurer["position"] != jewel["position"] and adventurer["position"] not in pits["inst"] and adventurer["position"] != [8, 4]:
        print("You got lost within the labyrinth.")
    # timedelta() represents time in units
    duration = timedelta(seconds=time.perf_counter()-starttime)
    print("Searching for the solution took: ", duration)
    return 0

main()

Type in enter to continue or Q to quit:

position currently being explored ->  [6, 0]
[[5, 8], [5, 6], [6, 7], [4, 7], [8, 5], [8, 3], [9, 4], [7, 4]]
1
[[6, 1]]
['X' 'X' 'X' 'X' 'X' '*' 'X' 'X' 'X' 'X']
['X' 'X' 'X' '.' 'X' '.' '.' '.' '.' 'X']
['X' 'X' '.' '.' '.' '.' 'X' 'X' '.' 'X']
['X' 'X' '.' 'X' 'X' 'X' 'X' '.' '.' 'X']
['X' 'X' '.' '.' '.' '.' '.' '.' 'X' 'X']
['X' 'X' '.' 'X' 'X' '.' 'X' 'P' 'X' 'X']
['A' '.' '.' '.' 'X' '.' 'X' '.' 'X' 'X']
['X' 'X' 'X' '.' 'X' 'X' 'X' '.' '.' 'X']
['X' 'X' 'X' '.' 'W' '.' 'X' 'X' '.' 'X']
['X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X' 'X']
Keep going!

position currently being explored ->  [6, 1]
[[5, 8], [5, 6], [6, 7], [4, 7], [8, 5], [8, 3], [9, 4], [7, 4]]
1
[[6, 2]]
['X' 'X' 'X' 'X' 'X' '*' 'X' 'X' 'X' 'X']
['X' 'X' 'X' '.' 'X' '.' '.' '.' '.' 'X']
['X' 'X' '.' '.' '.' '.' 'X' 'X' '.' 'X']
['X' 'X' '.' 'X' 'X' 'X' 'X' '.' '.' 'X']
['X' 'X' '.' '.' '.' '.' '.' '.' 'X' 'X']
['X' 'X' '.' 'X' 'X' '.' 'X' 'P' 'X' 'X']
['.' 'A' '.' '.' 'X' '.' 'X' '.'

0